In [2]:
#Bowerbird null model (all males are guarders)

#Key
#Notes- Paramters contain underscores, while functions don't
#SB: Stay at bower
#FG: Foraging
#a=-1: denotes a staying at bower action on ticket
#a=-2: denotes a foraging action on ticket
#a=-3: denotes a female visiting action on a ticket
#networkwriter: generates network where everybird is connected
#addtotimeline: adds tickets to timeline
#travel_times_linear_p: generates a matrix that contains the probabilities of travelling to a given a bower 
#if the relationship between distance and travel preference is linear
#improb: The probability of traveling improb_distance or less
#improb_distance: The distance at which there is only a (1-improb)% chance of choosing to travel
#lamb: lambda calculated by solving improb=1-np.exp(-lamb*improb_distance)
#bower_states: options are 1(bower intact) and 0 (bower destroyed-will never be the case when all guarders)
#male_states: options are 1 (male present at bower) and 0 (male absent from bower)
#fitness_states: keeps track of number of matings a given male has had
#t: time
#nodes: total number of male bowerbirds in the network




import math
import random
import numpy as np


# makes a ticket
def ticketgenerator(tau,t, o, a):
    ticket={
        'tau': tau,
        'time': t,
        'owner': o,
        'action': a,
    }
    return ticket;

#writes the edges to a network of n birds where everyone is connected to everyone else 
def networkwriter(n):
    connect=[0]*n
    for i in range(0, n): 
        a=list(range(n)) #generating a list that goes from 0 to n-1
        a.remove(i) #individual removes themselves from the network
        connect[i]=a #corresponds to the list of people person i is linked to
    return connect; #returns a list of lists with all the connections in the network

# function for determining the next time based on our rate parameters
def nexttau(rateParameter):
    taus=random.expovariate(rateParameter) #math.log is in base e, and random.random picks a number from range 0.0 to 1.
    return taus #generates n-length list, starting at index 0

#adds new tickets to timeline -- Stefano, any more efficient suggestions for keeping timeline in order?
def addtotimeline(tic, timeline):
    if not timeline:
        timeline.append(tic)
    else:
        ind=len(timeline)-1
        end=0
        while (tic['time']<timeline[ind]['time'] and end==0): #moves backwards until it finds where to place the ticket based on the listed times
            ind=ind-1 
            if(ind<0):
                end=1
        ind=ind+1
        timeline.insert(ind, tic)
    

def SBtickethandler (SB_tic, timeline,SB_param, t_max, male_states):
    ow=SB_tic['owner'] 
    male_states[ow]=1
    t=SB_tic['time']
    FG_time=nexttau(SB_param)+t #We use SB_param because the time represents when the bird leaves the bower
    if FG_time<t_max:
        FG_tic=ticketgenerator(FG_time-t,FG_time,ow,-2) #-2 denotes foraging action
        addtotimeline(FG_tic,timeline)

def FGtickethandler (FG_tic, timeline,FG_param, t_max, male_states):
    ow=FG_tic['owner'] 
    male_states[ow]=0
    t=FG_tic['time']
    SB_time=nexttau(FG_param)+t #We use FG_param because the time represents when the bird returns to the bower
    if SB_time<t_max:
        SB_tic=ticketgenerator(SB_time-t,SB_time,ow,-1) #-1 denotes staying at bower
        addtotimeline(SB_tic,timeline)
    
def FVtickethandler (FV_tic, timeline, FV_param, t_max, male_states, fitness_states, nodes, female_preferences, travel_times):
    ow=FV_tic['owner']
    t=FV_tic['time']
    if bower_states[ow]==1 and male_states[ow]==1: #if the bower is intact and the male is present
        fitness_states[ow]=fitness_states[ow]+1 #assumption: female always mates if bower is intact and male present
        new_FV_ow=np.random.choice(list(range(nodes)))
        new_FV_time=t+nexttau(FV_param)
        if new_FV_time<t_max:
            new_FV_tic=ticketgenerator(new_FV_time-t,new_FV_time, new_FV_ow, -3)
            addtotimeline(new_FV_tic, timeline)
    else: #if female does not successfully mate
        new_FV_ow=np.random.choice(list(range(nodes)), p=female_preferences[ow]) #choose a male based on preference (a function of distance)
        new_FV_time=t+travel_times[ow][new_FV_ow] #she goes directly to this male
        if new_FV_time<t_max:
            new_FV_tic=ticketgenerator(new_FV_time-t,new_FV_time, new_FV_ow, -3)
            addtotimeline(new_FV_tic, timeline)
        
    
#FOR FUTURE: Write a function that generates a probability network of n bowerbirds 



In [3]:
#for now, we write a simple example network with 4 bowers forming a square
nodes=4

#HOW TO REPRESENT INDIVIDUAL'S DIST FROM SELF? nan?
distances = np.array([[150.0]*nodes]*nodes) #(dist in m)initialize a nodes-by-nodes matrix (1st nrows, 2nd ncols)
for i in range(4):
    distances[i][i]=9999.0 #or math.inf but that makes the matrix disp ugly
for i in range(2):
    distances[i][i+2]=((2*(150**2)))**(1/2)
    distances[i+2][i]=((2*(150**2)))**(1/2)
print(distances)

bird_speed=12 #m/s
travel_times=np.array([[0.0]*nodes]*nodes)
for i in range(nodes):
    for j in range(nodes):
        travel_times[i][j]=distances[i][j]/bird_speed
print(travel_times) #right now the numbers on the diagonals aren't accurate, but we can't have the bird choosing to go to it's own node

#solve for lambda 
improb=0.99
improb_distance=800
lamb=-math.log(1-improb)/improb_distance

#will write female preference based on cumulative exponential decay (lambda=.00576)
female_preferences=np.array([[0.]*nodes]*nodes)

for i in range(nodes):
    for j in range(nodes):
        female_preferences[i][j]=math.exp(-lamb*distances[i][j])        
    female_preferences[i]=female_preferences[i]/sum(female_preferences[i])
print(female_preferences)


[[9999.          150.          212.13203436  150.        ]
 [ 150.         9999.          150.          212.13203436]
 [ 212.13203436  150.         9999.          150.        ]
 [ 150.          212.13203436  150.         9999.        ]]
[[833.25        12.5         17.67766953  12.5       ]
 [ 12.5        833.25        12.5         17.67766953]
 [ 17.67766953  12.5        833.25        12.5       ]
 [ 12.5         17.67766953  12.5        833.25      ]]
[[8.83582673e-26 3.70465006e-01 2.59069988e-01 3.70465006e-01]
 [3.70465006e-01 8.83582673e-26 3.70465006e-01 2.59069988e-01]
 [2.59069988e-01 3.70465006e-01 8.83582673e-26 3.70465006e-01]
 [3.70465006e-01 2.59069988e-01 3.70465006e-01 8.83582673e-26]]


In [4]:
#just a test

#Parameters:
t=0 #start at time 0
t_max=12
timeholder=0
timeline=[] #initialize the timeline
FV_param=0.3 #totally random
FG_param=1.0
SB_param=1.5
nodes=4
bower_states=[1]*nodes
male_states=[1]*nodes
fitness_states=[0]*nodes

    
first_male=random.choice(list(range(nodes))) #The lucky bowerbird that is chosen by the female first
first_female_time=t+nexttau(FV_param) #determine the time when the first female arrives
first_female_tic=ticketgenerator(first_female_time-t,first_female_time, first_male, -3) #first female ticket
addtotimeline(first_female_tic, timeline)

for i in range(nodes):
    init_FG_time=t+nexttau(FG_param)
    init_FG_tic=ticketgenerator(init_FG_time-t,init_FG_time, i, -2)
    addtotimeline(init_FG_tic, timeline)
        


In [5]:
#while loop

while t<t_max:
    #print(t)
    if timeholder>(len(timeline)-1): #if timeholder exceeds max timeline index
        #print(sum(ppl_state)/num_ppl)
        print('end')
        break
    next_tic=timeline[timeholder] #look at the ticket corresponding to where we are on the timeline
    if (next_tic['action']==-1): #if it's a cure ticket, use the corresponding handler
        SBtickethandler(next_tic,timeline,SB_param,t_max,male_states)
    elif next_tic['action']==-2: #if it's a time check ticket use the corresponding handler
        FGtickethandler(next_tic,timeline,FG_param,t_max,male_states)
    else: #if it's a spread ticket, use the corresponding handler
        FVtickethandler(next_tic, timeline, FV_param, t_max, male_states, fitness_states, nodes, female_preferences, travel_times)
    t=next_tic['time'] #new time based on the ticket we just read
    timeholder=timeholder+1 #increment timeholder

print(timeline)


end
[{'tau': 0.10535186452019435, 'time': 0.10535186452019435, 'owner': 1, 'action': -2}, {'tau': 0.20680333680502197, 'time': 0.20680333680502197, 'owner': 0, 'action': -2}, {'tau': 0.1240437814823741, 'time': 0.22939564600256845, 'owner': 1, 'action': -1}, {'tau': 0.5032184533142976, 'time': 0.5032184533142976, 'owner': 2, 'action': -2}, {'tau': 0.03113325489534058, 'time': 0.5343517082096382, 'owner': 2, 'action': -1}, {'tau': 0.5431469066579301, 'time': 0.7725425526604985, 'owner': 1, 'action': -2}, {'tau': 1.21909691916543, 'time': 1.753448627375068, 'owner': 2, 'action': -2}, {'tau': 1.0478282895486037, 'time': 1.8203708422091023, 'owner': 1, 'action': -1}, {'tau': 1.0737380620686514, 'time': 2.8941089042777537, 'owner': 1, 'action': -2}, {'tau': 1.2329911047800195, 'time': 2.9864397321550875, 'owner': 2, 'action': -1}, {'tau': 0.21814566000510283, 'time': 3.2045853921601903, 'owner': 2, 'action': -2}, {'tau': 3.0162332223911026, 'time': 3.2230365591961245, 'owner': 0, 'action': 

In [6]:
d={}


for i in range(nodes):
    d["tl{0}".format(i)]=[]
for j in range(len(timeline)):
    #for i in range(nodes):
     #   if timeline[j]['owner']==i:
    d["tl{0}".format(timeline[j]['owner'])].append(timeline[j])
    

print("tl0----")
print(d["tl0"])
print("tl1----")
print(d["tl1"])
print("tl2----")
print(d["tl2"])
print("tl3----")
print(d["tl3"])  

for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[0]*l
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            ta[j]=tl[j]['time']
    d["times{0}".format(i)]=ta

print("times0----")
print(d["times0"])
print("times1----")
print(d["times1"])
print("times2----")
print(d["times2"])
print("times3----")
print(d["times3"]) 

#append tmax - sumoftaus to each list (so it reaches tmax exactly)
for i in range(nodes):
    d["times{0}".format(i)].insert(0,0)
    d["times{0}".format(i)].append(t_max)


import csv
timess=[d["times0"],d["times1"],d["times2"],d["times3"]]
csvfile = "example_times2"
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(timess)
           
    

tl0----
[{'tau': 0.20680333680502197, 'time': 0.20680333680502197, 'owner': 0, 'action': -2}, {'tau': 3.0162332223911026, 'time': 3.2230365591961245, 'owner': 0, 'action': -1}, {'tau': 0.08700103509344048, 'time': 3.310037594289565, 'owner': 0, 'action': -2}, {'tau': 0.19095390621869424, 'time': 3.5009915005082592, 'owner': 0, 'action': -1}, {'tau': 0.592294666728725, 'time': 4.093286167236984, 'owner': 0, 'action': -2}, {'tau': 0.616278358559093, 'time': 4.709564525796077, 'owner': 0, 'action': -1}, {'tau': 1.3771024713915647, 'time': 6.086666997187642, 'owner': 0, 'action': -2}, {'tau': 0.018534054735423133, 'time': 6.105201051923065, 'owner': 0, 'action': -1}, {'tau': 1.4544652997830365, 'time': 7.559666351706102, 'owner': 0, 'action': -2}, {'tau': 0.7982227865385338, 'time': 8.357889138244635, 'owner': 0, 'action': -1}, {'tau': 0.6139376253531736, 'time': 8.971826763597809, 'owner': 0, 'action': -2}, {'tau': 0.19916929767004632, 'time': 9.170996061267855, 'owner': 0, 'action': -1},

In [ ]:
for i in range(nodes):
    tl=d["tl{0}".format(i)]
    l=len(tl)
    ta=[0]*l
    for j in range(l):
        if tl[j]['action']!=-3: #female visit taus don't mean anything for the male bc she isn't necessarily coming back to the same male
            ta[j]=tl[j]['tau']
    d["taus{0}".format(i)]=ta

print("taus0----")
print(d["taus0"])
print(sum(d["taus0"]))
print("taus1----")
print(d["taus1"])
print(sum(d["taus1"]))
print("taus2----")
print(d["taus2"])
print(sum(d["taus2"]))
print("taus3----")
print(d["taus3"]) 
print(sum(d["taus3"]))


#append tmax - sumoftaus to each list (so it reaches tmax exactly)
for i in range(nodes):
    d["taus{0}".format(i)].append(t_max-sum(d["taus{0}".format(i)]))

    
print(sum(d["taus0"]))
print(sum(d["taus1"]))
print(sum(d["taus2"]))
print(sum(d["taus3"]))  

In [9]:
#THINGS START GETTING MESSY FROM HERE ON OUT :/

from matplotlib import pyplot as plt
from itertools import cycle, islice
import pandas, numpy as np  # I find np.random.randint to be better

# Make the data
x = [{i: d["taus{0}".format(i)]} for i in range(nodes)]
print(x)
df = pandas.DataFrame(x)
print(df)
# Make a list by cycling through the colors you care about
# to match the length of your data.
my_colors = list(islice(cycle(['b', 'r', 'g', 'y', 'k']), None, len(df)))

# Specify this list of colors as the `color` option to `plot`.
df.plot(kind='bar', stacked=True, color=my_colors)


[{0: [1.5094725205907227, 2.7343286000917577, 2.0505258454219515, 0.7847714337193636, 0.43133465140531424, 0.9209602110339512, 0.7228287594172667, 0.6459907801285905, 0.31494284727925503, 0.4318438540619489, 0.5547057349169471, 0.8982947619329309]}, {1: [1.6561683466194403, 0.7223021033647803, 1.4422114166288669, 0, 1.9057849127210167, 0.293514435437384, 1.1666200346594522, 0.3139978808459878, 1.4066893771046356, 0.285568300402975, 0.37499164845652366, 0.22330212346373202, 0.1565456895529156, 0.08319408073660384, 1.6772674069287437, 0.2918422430769425]}, {2: [0.8408591536993193, 0.36649595928225653, 0.9223583538999933, 0.2223690897980588, 0, 0.6493124680517246, 0.5787060369599835, 0.20280126312164803, 1.6889034132932195, 0.03304917878649505, 0.4756188329581814, 1.8683091060370396, 0.9372739308345102, 0.856027368467636, 0.9745447162815069, 0.28977709438828825, 0.8611451415228153, 0.23244889261732382]}, {3: [2.2168576323523785, 0.19776713851980832, 0.1157916609097569, 0.44276855733341813

TypeError: Empty 'DataFrame': no numeric data to plot

In [7]:
#Create travel_time__linear_p (might use later)
travel_times_linear_p=np.array([[0.0]*nodes]*nodes)
for i in range(nodes):
    for j in range(nodes):
        if i==j: #if the row and the column index are the equal
            travel_times[i][j]=0 #There is 0 probability they will travel to their own bower when choosing to fly to another bower
        else: 
            if i==0: #if on the first row of the probability matrix
                travel_times_linear_p[i][j]=travel_times[i][1]/travel_times[i][j] #skip the first element in the row (because it equals 0) and compare probabilities of other elements in the row based on travel distance (negative linear relationship btwn distance and prob)
            else:
                travel_times_linear_p[i][j]=travel_times[i][0]/travel_times[i][j] #use the first element in a row and compare probabilities of other elements in the row based on distance (negative linear relationship btwn distance and prob)
    travel_times_linear_p[i]=travel_times_linear_p[i]/sum(travel_times_linear_p[i]) #normalize the probabilities so they add up to 1
print(travel_times_linear_p)



[[0.         0.36939806 0.26120387 0.36939806]
 [0.36939806 0.         0.36939806 0.26120387]
 [0.26120387 0.36939806 0.         0.36939806]
 [0.36939806 0.26120387 0.36939806 0.        ]]


In [8]:
print(fitness_states)

[0, 0, 1, 0]
